In [1]:
import json
import sys
import requests
from datetime import datetime
from dateutil import tz
from pprint import pprint
from collections import namedtuple

In [ ]:
TeamVenueRecord = namedtuple('TeamVenueRecord', 'city, name, timezone, address')
NHLTeamInfoRecord = namedtuple('NHLTeamInfoRecord', 'conference, division')
TeamRecord = namedtuple('TeamRecord', 'id, abbreviation, name, active, city, fullname, creation_year, website, venue league_info')
NHLTeamStandingRecord = namedtuple('NHLTeamStandingRecord', 'pts, win, lose, ot, games_played, goals_against, goals_scored, division_rank, conference_rank, wildcard_rank, league_rank')

In [2]:
class PowerDict(object):
    
    def __init__(self):
        self._data = {}

    def keys(self):
        return self._data.keys()
    
    @property
    def data(self):
        #d = {}
        #for k in self._data.keys():
        #    if isinstance(self._data[k], PowerDict):
        #        d[k] = self._data[k].data
        #    else:
        #        d[k] = self._data[k]
        return self._data

    def __getitem__(self, key):
        return self._data[key]

    def __setitem__(self, key, value):
        self._data[key] = value

    def __setattr__(self, name, value):
        if name != '_data':   
            self._data[name] = value
        else:
            return object.__setattr__(self, name, value)

    def __getattr__(self, key):
        return self._data[key]
    
    def __repr__(self):
        return self._data.__repr__()
    
    def __str__(self):
        return self._data.__str__()

In [ ]:
p = PowerDict()
p.test = 45
p.test
p['test'] = 55
p['test']

In [3]:
class Team(PowerDict):
    
    def __init__(self, id=0, abbreviation='', name='', fullname='', city='', active=False, creation_year=0, website='', venue=None, league_info=None):
        team = {}
        team['id'] = id
        team['abbreviation'] = abbreviation
        team['name'] = name
        team['fullname'] = fullname
        team['city'] = city
        team['active'] = active
        team['creation_year'] = creation_year
        team['website'] = website
        team['venue'] = venue
        team['league_info'] = league_info
        self._data = team
        
class TeamVenue(PowerDict):
    
    def __init__(self, city='', name='', timezone='', address=''):
        venue = {}
        venue['city'] = city
        venue['name'] = name
        venue['timezone'] = timezone
        venue['address'] = address
        self._data = venue

class Standing(PowerDict):
    
    def __init__(self, team_id, pts, win, losses, ot, games_played, goals_against, goals_scored, ranks, extra_info={}):
        standing = {}
        standing['team_id'] = team_id
        standing['pts'] = pts
        standing['win'] = win
        standing['losses'] = losses
        standing['ot'] = ot
        standing['games_played'] = games_played
        standing['goals_against'] = goals_against
        standing['goals_scored'] = goals_scored
        standing['ranks'] = ranks
        standing['extra_info'] = extra_info
        self._data = standing

In [ ]:
p = PowerDict()
p.test = 12
te = Team(1, 'ABC', 'Super ABC', 'City Super ABC', 'City')
#print(te.city)
te.city = 9
te['city'] = 'Test'
#print(te.city)
#print(te['city'])
#print(te)
m = Matchup('a1', 1)
m['schedule'] = []
m.schedule = []
m.away = 9
print(m)

In [4]:
class MatchupResult(PowerDict):
    
    def __init__(self, home_win=0, away_win=0):
        results = {}
        results['home_win'] = home_win
        results['away_win'] = away_win
        self._data = results

class MatchupSeasonResults(PowerDict):

    def __init__(self, home_win, away_win, games=[]):
        results = {}
        results['home_win'] = home_win
        results['away_win'] = away_win
        results['games'] = games
        self._data = results

class Game(PowerDict):
    GAME_STATE_SCHEDULED = 1
    GAME_STATE_IN_PROGRESS = 2
    GAME_STATE_SCHEDULED = 3
    GAME_STATE_FINISHED = 4

    def __init__(self, home, away, date='', state=GAME_STATE_SCHEDULED, home_goal=0, away_goal=0, extra_data=None):
        game = {}
        game['home'] = home
        game['away'] = away
        game['date'] = date
        game['state'] = state
        game['home_goal'] = home_goal
        game['away_goal'] = away_goal
        game['extra_data'] = extra_data
        self._data = game

class Matchup(PowerDict):
    
    def __init__(self, id, round, home=0, away=0, start='', schedule={}, season_results={}, results=None):
        matchup = {}
        matchup['id'] = id
        matchup['home'] = home
        matchup['away'] = away
        matchup['round'] = round
        matchup['start'] = start
        matchup['schedule'] = schedule
        matchup['season_results'] = season_results
        matchup['results'] = results
        self._data = matchup
        
    @property
    def started(self):
        return False

In [5]:
GAME_STATE_SCHEDULED = 1
GAME_STATE_IN_PROGRESS = 2
GAME_STATE_SCHEDULED = 3
GAME_STATE_FINISHED = 4


class PoolData(PowerDict):
    def __init__(self, year):
        self._data = {}
        self.year = year
        self.teams = []
        self.players = []
        self.years = []

    def create_team(self, id, abbreviation, name, fullname, city, active=False, creation_year=0, website='', venue=None, league_info=None):
        return Team(id, abbreviation, name, fullname, city, active, creation_year, website, venue, league_info)

    def create_team_venue(self, city, name='', timezone='', address=''):
        return TeamVenue(city, name, timezone, address)

    def create_standings(self, team_id, pts, win, losses, ot, games_played, goals_against, goals_scored, ranks, extra_info={}):
        return Standing(team_id, pts, win, losses, ot, games_played, goals_against, goals_scored, ranks, extra_info)

    def create_matchup_results(self, home_win=0, away_win=0):
        return MatchupResult(home_win, away_win)

    def create_matchup_season_results(self, home_win, away_win, games=[]):
        return MatchupSeasonResults(home_win, away_win, games)

    def create_matchup_game(self, home, away, date='', state=GAME_STATE_SCHEDULED, home_goal=0, away_goal=0, extra_data=None):
        return Game(home, away, date, state, home_goal, away_goal, extra_data)

    def create_matchup(self, id, round, home=0, away=0, start='', schedule={}, season_results={}, results=None):
        if not results:
            results = results = self.create_matchup_results()
        return Matchup(id, round, home, away, start, schedule, season_results, results)

In [6]:
class NHLData(object):
    
    def __init__(self, year):
        self._year = year
        self._teams = None
        self._standings = None
    
    def get_team(self, id):
        url = 'https://statsapi.web.nhl.com/api/v1/teams/' + str(id)
        team = requests.get(url).json()
        return team['teams'][0]
    
    def get_standings(self):
        if not self._standings:
            ystr = str(self._year) + str(self._year + 1)
            url = 'https://statsapi.web.nhl.com/api/v1/standings?season=' + ystr
            standings = requests.get(url).json()
            self._standings = standings["records"]
        return self._standings

    def get_schedule(self, team):
        print('Get schedule for ' + str(team))
        url = 'https://statsapi.web.nhl.com/api/v1/schedule?startDate=' + str(self._year) + '-10-01&endDate=' + str(self._year+1) + '-06-29&expand=schedule.teams,schedule.linescore,schedule.broadcasts,schedule.ticket,schedule.game.content.media.epg&leaderCategories=&site=en_nhlCA&teamId=' + str(team)
        team_schedule = requests.get(url)
        return team_schedule.json()

    def get_playoff_schedule(self, team):
        url = 'https://statsapi.web.nhl.com/api/v1/schedule?startDate=' + str(self._year+1) + '-04-01&endDate=' + str(self._year+1) + '-06-29&expand=schedule.teams,&site=en_nhlCA&teamId=' + str(team)
        team_schedule = requests.get(url)
        return team_schedule.json()

    def get_teams(self):
        if not self._teams:
            standings = self.get_standings()
            teams = {}
            for record in standings:
                for team in record['teamRecords']:
                    info = self.get_team(team['team']['id'])
                    team_record = {'info': info, 'standings': team, 'schedule': []}
                    teams[team['team']['id']] = team_record
            self._teams = teams
        return self._teams


class NHLDataConverter(object):
     
    def __init__(self, nhl_data, pool_data):
        self._nhl_data = nhl_data
        self._pool_data = pool_data

    def extract_team_info(self, team):
        info = {}
        info['abbreviation'] = team['abbreviation']
        info['name'] = team['teamName']
        info['active'] = team['active']
        info['city'] = team['locationName']
        info['fullname'] = team['name']
        info['creation_year'] = team['firstYearOfPlay']
        info['website'] = team['officialSiteUrl']
        return info
        
    def extract_team_venue(self, team):
        venue_city = team['venue']['city']
        venue_name = team['venue']['name']
        venue_timezone = team['venue']['timeZone']['id']
        return self._pool_data.create_team_venue(venue_city, venue_name, venue_name)

    def extract_team_league_info(self, team):
        conference_id = team['conference']['id']
        conference_name = team['conference']['name']
        division_id = team['division']['id']
        division_name = team['division']['name']
        return self.create_nhl_team_info(conference_id, conference_name, division_id, division_name)

    def extract_standing_info(self, team_standing):
        info = {}
        info['team_id'] = int(team_standing['team']['id'])
        info['pts'] = int(team_standing['points'])
        info['win'] = int(team_standing['leagueRecord']['wins'])
        info['losses'] = int(team_standing['leagueRecord']['losses'])
        info['ot'] = int(team_standing['leagueRecord']['ot'])
        info['games_played'] = int(team_standing['gamesPlayed'])
        info['goals_against'] = int(team_standing['goalsAgainst'])
        info['goals_scored'] = int(team_standing['goalsScored'])
        return info

    def extract_ranks(self, team_standing):
        ranks = {}
        league_rank = int(team_standing['leagueRank'])
        conference_rank = int(team_standing['conferenceRank'])
        division_rank = int(team_standing['divisionRank'])
        wildCard_rank = int(team_standing['wildCardRank'])
        return self.create_nhl_ranks(league_rank, conference_rank, division_rank, wildCard_rank)

    def create_nhl_team_info(self, conference_id, conference_name, division_id, division_name):
        info = {}
        info['conference'] = {'id': conference_id, 'name': conference_name}
        info['division'] = {'id': division_id, 'name': division_name}
        return info

    def create_nhl_ranks(self, league_rank, conference_rank, division_rank, wildCard_rank):
        rank = {}
        rank['league_rank'] = league_rank
        rank['conference_rank'] = conference_rank
        rank['division_rank'] = division_rank
        rank['wildCard_rank'] = wildCard_rank
        return rank

    def get_teams(self):
        nhl_teams = self._nhl_data.get_teams() 
        db_teams = {}
        for tid in nhl_teams:
            t = nhl_teams[tid]['info']
            
            info = self.extract_team_info(t)
            venue = self.extract_team_venue(t)
            league_info = self.extract_team_league_info(t)
            team = self._pool_data.create_team(tid, info['abbreviation'], info['name'], info['fullname'], 
                                               info['city'], info['active'], info['creation_year'], 
                                               info['website'], venue, league_info)
            db_teams[tid] = team
        return db_teams
    
    def get_standings(self):
        nhl_standings = self._nhl_data.get_standings()
        db_standings = {}
        for division in nhl_standings:
            for team_standing in division['teamRecords']:
                info = self.extract_standing_info(team_standing)
                ranks = self.extract_ranks(team_standing)
                standing = self._pool_data.create_standings(info['team_id'], info['pts'], info['win'], 
                                            info['losses'], info['ot'], info['games_played'], 
                                            info['goals_against'], info['goals_scored'], ranks)
                db_standings[info['team_id']] = standing
        return db_standings

    def get_data(self):
        data = {}

In [7]:
nhl_extractor = NHLDataConverter(NHLData(2016), PoolData(2016))
teams = nhl_extractor.get_teams()
standings = nhl_extractor.get_standings()

In [8]:
class MatchupTree(object):
    STATE_UNITIALIZED = 1
    STATE_NOT_STARTED = 2
    STATE_STARTED     = 3
    STATE_FINISHED    = 4
    
    def __init__(self):
        self._nodes = {}
        self._on_matchup_finished = None
        self._on_round_finished = None
    
    def create_node(self, id, round, right=None, left=None, next=None, matchup=None, state=STATE_UNITIALIZED):
        self._nodes[id] = MatchupTreeNode(id, round, right, left, next, matchup, state)
    
    def update_node_state(self, id, state):
        self._nodes[id]['state'] = state
        
    def update_node_links(self, id, right=None, left=None, next=None):
        if right:
            self._nodes[id]['right'] = right
        if left:
            self._nodes[id]['left'] = left
        if next:
            self._nodes[id]['next'] = next
    
    def get_started_nodes(self):
        started = []
        for node in self._nodes.values():
            if node['state'] == MatchupTree.STATE_STARTED:
                started.append(node)
        return started

    def keys(self):
        return self._nodes.keys()

    def __getitem__(self, key):
        return self._nodes[key]
    
    def __getattr__(self, key):
        return self._nodes[key]
    
class MatchupTreeNode(PowerDict):

    def __init__(self, id, round, right=None, left=None, next=None, matchup=None, state=MatchupTree.STATE_UNITIALIZED):
        node = {}
        node['id'] = id
        node['round'] = round
        node['state'] = state
        node['right'] = right
        node['left'] = left
        node['next'] = next
        node['matchup'] = matchup
        self._data = node

In [ ]:
tree = MatchupTree()

In [ ]:
#tree.create_node('w1', 1)
#tree.w1['state'] = MatchupTree.STATE_STARTED
pprint(tree.get_started_nodes())

In [9]:
db_standings = {'Eastern':{'Atlantic':[], 'Metropolitan':[], 'teams':[]},
             'Western':{'Central':[], 'Pacific':[], 'teams':[]},
             'teams':[]}

league = sorted(standings.values(), key=lambda k: int(k.ranks['division_rank']))
for team in league:
    db_standings['teams'].append(team)
    id = team.team_id
    team_info = teams[id]
    db_standings[team_info.league_info['conference']['name']]['teams'].append(team)
    db_standings[team_info.league_info['conference']['name']][team_info.league_info['division']['name']].append(team)
db_standings['teams'] = sorted(db_standings['teams'], key=lambda k: int(k.ranks['league_rank']))

db_standings['Eastern']['teams'] = sorted(db_standings['Eastern']['teams'], key=lambda k: int(k.ranks['conference_rank']))
db_standings['Western']['teams'] = sorted(db_standings['Western']['teams'], key=lambda k: int(k.ranks['conference_rank']))

db_standings['Eastern']['Atlantic'] = sorted(db_standings['Eastern']['Atlantic'], key=lambda k: int(k.ranks['division_rank']))
db_standings['Eastern']['Metropolitan'] = sorted(db_standings['Eastern']['Metropolitan'], key=lambda k: int(k.ranks['division_rank']))
db_standings['Western']['Central'] = sorted(db_standings['Western']['Central'], key=lambda k: int(k.ranks['division_rank']))
db_standings['Western']['Pacific'] = sorted(db_standings['Western']['Pacific'], key=lambda k: int(k.ranks['division_rank']))

In [ ]:
pprint(standings)

In [ ]:
create_matchup('w1', 1, results=create_matchup_result(4,3))

In [10]:
tree = MatchupTree()
tree.create_node('sc', 4, next=None)

tree.create_node('e', 3, next=tree.sc)
tree.create_node('w', 3, next=tree.sc)

tree.create_node('a', 2, next=tree.e)
tree.create_node('m', 2, next=tree.e)
tree.create_node('c', 2, next=tree.w)
tree.create_node('p', 2, next=tree.w)

tree.create_node('a1', 1, next=tree.a)
tree.create_node('a2', 1, next=tree.a)
tree.create_node('m1', 1, next=tree.m)
tree.create_node('m2', 1, next=tree.m)
tree.create_node('c1', 1, next=tree.c)
tree.create_node('c2', 1, next=tree.c)
tree.create_node('p1', 1, next=tree.p)
tree.create_node('p2', 1, next=tree.p)

tree.update_node_links('sc', tree.e, tree.w)

tree.update_node_links('w', tree.p, tree.c)
tree.update_node_links('e', tree.m, tree.a)

tree.update_node_links('c', tree.c2, tree.c1)
tree.update_node_links('p', tree.p2, tree.p1)
tree.update_node_links('a', tree.a2, tree.a1)
tree.update_node_links('m', tree.m2, tree.m1)


In [30]:
def display(tree):
    nb_round = 4
    width = (nb_round * 2) - 1
    heigh = (2**(nb_round-1)) -1
    display = [['' for x in range(width)] for y in range(heigh)] 
    def walk_matchup_tree(root, x, y, dx):
        display[x][y] = root.id
        if root.left is not None:
            walk_matchup_tree(root.left, x+dx, y-(root.round - 1), dx)
        if root.right is not None:
            walk_matchup_tree(root.right, x+dx, y+(root.round - 1), dx)

    display[3][2] = 'sc'
    walk_matchup_tree(tree.w, 2, 3, -1)
    walk_matchup_tree(tree.e, 4, 3, 1)

    for y in range(7):
        for x in range(7):
            id = display[x][y]
            if id != '':
                node = tree[id]
                matchup = node.matchup
                if not matchup:
                    sys.stdout.write('{0:15}'.format(id))
                else:
                    home = teams[matchup.home].abbreviation
                    away = '?'
                    if matchup.away != 0:
                        away = teams[matchup.away].abbreviation
                    sys.stdout.write('{0:3}-{2} vs {3}-{1:3}'.format(home,away,matchup.results.home_win,matchup.results.away_win))
            else:
                sys.stdout.write('{0:15}'.format(id))
        sys.stdout.write('\n')
        
display(tree)

CHI-0 vs 4-NSH                                                                           MTL-2 vs 4-NYR
               STL-2 vs 4-NSH                                             NYR-2 vs 4-OTT               
MIN-1 vs 4-STL                              PIT-4 vs 2-NSH                              OTT-4 vs 2-BOS
                              ANA-2 vs 4-NSH               PIT-4 vs 3-OTT                              
ANA-4 vs 0-CGY                                                                           WSH-4 vs 2-TOR
               ANA-4 vs 3-EDM                                             WSH-3 vs 4-PIT               
EDM-4 vs 2-SJS                                                                           PIT-4 vs 1-CBJ


In [12]:
ealeader = db_standings['Eastern']['Atlantic'][0]
emleader = db_standings['Eastern']['Metropolitan'][0]
wcleader = db_standings['Western']['Central'][0]
wpleader = db_standings['Western']['Pacific'][0]
for team in db_standings['Eastern']['teams']:
    if int(team.ranks['wildCard_rank']) == 1:
        e1wild = team
    if int(team.ranks['wildCard_rank']) == 2:
        e2wild = team

for team in db_standings['Western']['teams']:
    if int(team.ranks['wildCard_rank']) == 1:
        w1wild = team
    if int(team.ranks['wildCard_rank']) == 2:
        w2wild = team

if int(ealeader.ranks['conference_rank']) < int(emleader.ranks['conference_rank']):
    tree.a1['matchup'] = nhl_extractor._pool_data.create_matchup('a1', 1, ealeader.team_id, e2wild.team_id)
    tree.m1['matchup'] = nhl_extractor._pool_data.create_matchup('m1', 1, emleader.team_id, e1wild.team_id)
else:
    tree.a1['matchup'] = nhl_extractor._pool_data.create_matchup('a1', 1, ealeader.team_id, e1wild.team_id)
    tree.m1['matchup'] = nhl_extractor._pool_data.create_matchup('m1', 1, emleader.team_id, e2wild.team_id)

tree.a2['matchup'] = nhl_extractor._pool_data.create_matchup('a2', 1, db_standings['Eastern']['Atlantic'][1].team_id, db_standings['Eastern']['Atlantic'][2].team_id)
tree.m2['matchup'] = nhl_extractor._pool_data.create_matchup('m2', 1, db_standings['Eastern']['Metropolitan'][1].team_id, db_standings['Eastern']['Metropolitan'][2].team_id)


if int(wcleader.ranks['conference_rank']) < int(wpleader.ranks['conference_rank']):
    tree.c1['matchup'] = nhl_extractor._pool_data.create_matchup('c1', 1, wcleader.team_id, w2wild.team_id)
    tree.p1['matchup'] = nhl_extractor._pool_data.create_matchup('p1', 1, wpleader.team_id, w1wild.team_id)
else:
    tree.c1['matchup'] = nhl_extractor._pool_data.create_matchup('c1', 1, wcleader.team_id, w1wild.team_id)
    tree.p1['matchup'] = nhl_extractor._pool_data.create_matchup('p1', 1, wpleader.team_id, w2wild.team_id)

tree.c2['matchup'] = nhl_extractor._pool_data.create_matchup('c2', 1, db_standings['Western']['Central'][1].team_id, db_standings['Western']['Central'][2].team_id)
tree.p2['matchup'] = nhl_extractor._pool_data.create_matchup('p2', 1, db_standings['Western']['Pacific'][1].team_id, db_standings['Western']['Pacific'][2].team_id)


In [13]:
def update_matchup(matchup):
    if not matchup['start']:
        # initial config
        nhl_extractor._nhl_data.get_playoff_schedule(matchup['home'])
    else:
        nhl_extractor._nhl_data.get_playoff_schedule(matchup['home'])

In [14]:
def extract_matchup_games(matchup):
    statuscode = {}
    statuscode[1] = GAME_STATE_SCHEDULED # 'Scheduled'
    statuscode[2] = GAME_STATE_SCHEDULED # 'Pre-Game'
    statuscode[3] = GAME_STATE_IN_PROGRESS # 'In Progress'
    statuscode[4] = GAME_STATE_IN_PROGRESS # 'In Progress - Critical'
    statuscode[5] = GAME_STATE_IN_PROGRESS # 'Game Over'
    statuscode[6] = GAME_STATE_IN_PROGRESS # 'Final'
    statuscode[7] = GAME_STATE_FINISHED # 'Final'
    schedules = nhl_extractor._nhl_data.get_playoff_schedule(matchup['home'])
    games = []
    for d in schedules['dates']:
        game = d['games'][0]
        home = game['teams']['home']['team']['id']
        home_score = game['teams']['home']['score']
        if game['gameType'] == 'P':
            if home != matchup['home']:
                away = home
                away_score = game['teams']['home']['score']
                home = game['teams']['away']['team']['id']
                home_score = game['teams']['away']['score']
            else:
                away = game['teams']['away']['team']['id']
                away_score = game['teams']['away']['score']
            #print(home, away, matchup['away'])
            if away == matchup['away']:
                date = game['gameDate']
                status = int(game['status']['statusCode'])
                pool_game = nhl_extractor._pool_data.create_matchup_game(home, away, date, statuscode[status], home_score, away_score)
                games.append(pool_game)
                #pprint(game)
                # print(status, home, home_score, away, away_score, date)
    return games

In [27]:
# Update schedule for active matchup
for id in tree.keys():
    node = tree[id]
    matchup = node.matchup
    if matchup and node.state != MatchupTree.STATE_FINISHED:
        print('Update schedule for ', id)
        matchup.schedule = extract_matchup_games(matchup) 

('Update schedule for ', 'sc')


In [28]:
# Update results for active matchup
for id in tree.keys():
    matchup = tree[id].matchup
    if matchup:
        matchup.results.home_win = 0
        matchup.results.away_win = 0
        for game in matchup.schedule:
            if game.state == GAME_STATE_FINISHED:
                if game.home_goal > game.away_goal:
                    matchup.results.home_win = matchup.results.home_win + 1
                elif game.away_goal > game.home_goal:
                    matchup.results.away_win = matchup.results.away_win + 1

In [29]:
# Update status for active matchup
for id in tree.keys():
    node = tree[id]
    matchup = node.matchup
    if matchup and node.state != MatchupTree.STATE_FINISHED:
        # Detect winner of matchup
        winner = 0
        if matchup.results.home_win == 4:
            winner = matchup.home
        elif matchup.results.away_win == 4:
            winner = matchup.away
        if winner:
            print('{0} Win'.format(teams[winner].name))
            node.state = MatchupTree.STATE_FINISHED
            if not node.next.matchup:
                node.next.matchup = nhl_extractor._pool_data.create_matchup(node.next.id, node.round+1, winner)
            else:
                node.next.matchup.away = winner
                # Need to order teams!!!!!!!
                home = standings[node.next.matchup.home]
                away = standings[node.next.matchup.away]
                if away.ranks['conference_rank'] < home.ranks['conference_rank']:
                    t = node.next.matchup.home
                    node.next.matchup.home = node.next.matchup.away
                    node.next.matchup.away = t
                    

Penguins Win


AttributeError: 'NoneType' object has no attribute 'matchup'

In [ ]:
tjson = json.dumps(teams, cls=PowerDictEncoder)

In [ ]:
ltjson = json.loads(tjson, cls=PowerDictDecoder)

In [ ]:
ltjson['5'].venue

In [ ]:
class PowerDictEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, PowerDict):
            result = obj.data
            result['__objectname__'] = type(obj).__name__
            return result
            # Let the base class default method raise the TypeError
        return json.JSONEncoder.default(self, obj)
    
class PowerDictDecoder(json.JSONDecoder):
    def __init__(self, *args, **kwargs):
        json.JSONDecoder.__init__(self, object_hook=self.object_hook, *args, **kwargs)

    def object_hook(self, obj):
        if '__objectname__' not in obj:
            return obj
        type = obj['__objectname__']
        newobj = globals()[type]()
        newobj._data = obj
        del newobj._data['__objectname__']
        return newobj

In [ ]:
datestr = "2017-04-10T00:41:45Z"
from_zone = tz.gettz('UTC')
to_zone = tz.gettz('America/New_York')
utc = datetime.strptime(datestr, '%Y-%m-%dT%H:%M:%SZ')
utc = utc.replace(tzinfo=from_zone)
date = utc.astimezone(to_zone)

In [ ]:
date.astimezone(from_zone).strftime('%Y-%m-%dT%H:%M:%SZ')
#date.strftime('%Y-%m-%dT%H:%M:%S%z')

In [ ]:
def now():
    to_zone = tz.gettz('America/New_York')
    return datetime.now(tz.tzlocal()).astimezone(to_zone)
def date(year, month, day, hour=0, minute=0, second=0):
    to_zone = tz.gettz('America/New_York')
    return datetime(year, month, day, hour, minute, second, tzinfo=to_zone)
def date(year, month, day, hour=0, minute=0, second=0):
    to_zone = tz.gettz('America/New_York')
    return datetime(year, month, day, hour, minute, second, tzinfo=tz.tzlocal()).astimezone(to_zone)

In [ ]:
n = now()

In [ ]:
d = date(2017, 2, 20, 10, 22, 33)

In [ ]:
type(d).__name__

In [ ]:
globals()['Team']

In [ ]:
t = PoolData(2017)

In [ ]:
t.team